In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

import torchvision

from torchvision import datasets, transforms

import torch.nn  as nn
import torch.optim as optim

import seaborn as sns

In [2]:
import mlflow


In [3]:
import mlflow.pytorch

# Params

In [5]:
class Params(object):
    def __init__(self, batch_size, epochs, seed, log_interval):
        self.batch_size = batch_size
        self.epochs = epochs
        self.seed = seed
        self.log_interval = log_interval
args = Params(256, 4, 0, 20)

# Dataset

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = datasets.MNIST('../data', train = True, download = True, transform= transform)

testset = datasets.MNIST('../data', train = False, download= True, transform = transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size= args.batch_size, shuffle=True)

test_loader= torch.utils.data.DataLoader(testset, batch_size= args.batch_size, shuffle= False)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...


/opt/conda/conda-bld/pytorch_1587428266983/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


# Model

In [8]:
class Model(nn.Module):
    def __init__(self,nH =32):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(
        nn.Linear(784, nH),
        nn.ReLU(),
        nn.Linear(nH,10)
        )
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [10]:
def train(epoch):
    model.train()
    for batch_id, data in enumerate(train_loader):
        inputs, labels = data
        opt.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if batch_id % args.log_interval == 0:
            pos = epoch * len(train_loader) + batch_id
            mlflow.log_metric('train_loss', loss.data.item()/ len(inputs)*1000)
            
            print('Train Epoch: {} [{} / {} ({:.0f}%)]\tLoss: {:.3f}'.format(
                epoch, batch_id * len(inputs), len(train_loader.dataset), 
                100. * batch_id / len(train_loader), loss.data.item()))
                  

In [18]:
def test(epoch):
    model.eval()
    test_loss = 0
    correct =0
    confusion_matrx = np.zeros([10, 10])
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            test_loss += loss_fn(outputs, labels).data.item()
            pred = outputs.data.max(1)[1]
            correct += pred.eq(labels.data).sum().item()
            
            
            for x,y in zip(pred.numpy(), labels.numpy()):
                confusion_matrx[x][y] += 1
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100.0 * correct / len(test_loader.dataset)
        
        
        pos = (epoch+1) * len(train_loader)
        mlflow.log_metric('test_loss', test_loss*1000)
        mlflow.log_metric('test_accuracy', test_accuracy)
        
        print('\n Test Set: Average Logss: {:.4f}, Accuracy: {}/ {} ({:.0f}%)\n'.
             format(test_loss, correct, len(test_loader.dataset), test_accuracy))
        
        if epoch == args.epochs:
            classes = np.arange(10)
            fig, ax = plt.subplots()
            im = ax.imshow(confusion_matrx, interpolation ='nearest', cmap=plt.cm.Blues)
            ax.figure.colorbar(im, ax = ax)
            ax.set(xticks=np.arange(confusion_matrx.shape[1]),
                  yticks = np.arange(confusion_matrx.shape[0]),
                  ylabel = 'True lable',
                  xlabel ='Pred Label',
                  title = 'Rpoch %d'% epoch)
            thresh = confusion_matrx.max()/2.
            for i in range(confusion_matrx.shape[0]):
                for j in range(confusion_matrx.shape[1]):
                    ax.text(j, i, int(confusion_matrx[i, j]),
                           ha="center", va ="center",
                           color="white" if confusion_matrx[i, j] > thresh else "black")
                    
            fig.tight_layout()
            image_path = '/home/prajin/Desktop/Pyotrch/images/%s.png'%(expt_id)
            plt.savefig(image_path)
            mlflow.log_artifact(image_path)

In [ ]:
for lr in [0.01, 0.05, 0.1]:
    for momentum in [0.9, 0.92, 0.95]:
        for hidden_nodes in [16 ,32, 48]:
            expt_id = '%d_%d_%d'%(int(lr*100), int(momentum*100), hidden_nodes)
            
            print('\nLR = %.2f, Momentum = %.2f, Hidden Nodes = %d\n'%(lr, momentum, hidden_nodes))
            model = Model(hidden_nodes)
            loss_fn = nn.CrossEntropyLoss()
            opt = optim.SGD(model.parameters(), lr = lr, momentum=momentum)
            with mlflow.start_run() as run:
                for key, value in vars(args).items():
                    mlflow.log_param(key, value)
                mlflow.log_param('lr', lr)
                mlflow.log_param('Momentum', momentum)
                mlflow.log_param('HiddnNode', hidden_nodes)
                
                
                for epoch in range(1, args.epochs +1):
                    train(epoch)
                    test(epoch)
                    
                    mlflow.pytorch.log_model(model, "models")


LR = 0.01, Momentum = 0.90, Hidden Nodes = 16

Train Epoch: 1 [0 / 60000 (0%)]	Loss: 2.310
Train Epoch: 1 [5120 / 60000 (9%)]	Loss: 1.214
Train Epoch: 1 [10240 / 60000 (17%)]	Loss: 0.571
Train Epoch: 1 [15360 / 60000 (26%)]	Loss: 0.616
Train Epoch: 1 [20480 / 60000 (34%)]	Loss: 0.353
Train Epoch: 1 [25600 / 60000 (43%)]	Loss: 0.323
Train Epoch: 1 [30720 / 60000 (51%)]	Loss: 0.430
Train Epoch: 1 [35840 / 60000 (60%)]	Loss: 0.397
Train Epoch: 1 [40960 / 60000 (68%)]	Loss: 0.411
Train Epoch: 1 [46080 / 60000 (77%)]	Loss: 0.330
Train Epoch: 1 [51200 / 60000 (85%)]	Loss: 0.317
Train Epoch: 1 [56320 / 60000 (94%)]	Loss: 0.326

 Test Set: Average Logss: 0.0012, Accuracy: 9148/ 10000 (91%)

Train Epoch: 2 [0 / 60000 (0%)]	Loss: 0.328
Train Epoch: 2 [5120 / 60000 (9%)]	Loss: 0.244
Train Epoch: 2 [10240 / 60000 (17%)]	Loss: 0.323
Train Epoch: 2 [15360 / 60000 (26%)]	Loss: 0.275
Train Epoch: 2 [20480 / 60000 (34%)]	Loss: 0.342
Train Epoch: 2 [25600 / 60000 (43%)]	Loss: 0.276
Train Epoch: 2 [3072

FileNotFoundError: [Errno 2] No such file or directory: '/home/prajin/Desktop/Pyotrch/images/1_90_16.png'